En este apartado se hicieron todos las tranformaciones necesarias para las funciones que va a consumir la API. Se tienen como fuentes de datos los archivos ya limpios: user_reviews_cleaned, steam_games_cleaned, user_items_cleaned. Se persigue la eficiencia y ligereza de estas consultas para facilitar el trabajo de la API.# Se i

In [22]:
# Se importan las librerías
import pandas as pd # Pandas para  trabajar con dataframe
import ast # Para la evaluación de líneas literales de python

In [ ]:
# Se cargan los archivos limpios en un dataframe
df_Steam_Games =pd.read_csv('..\\..\\Archivos Limpios\\output_steam_games_limpio.csv') # Se carga en el dataframe nuestro archivo de games
df_Users_Items =pd.read_csv('..\\..\\Archivos Limpios\\australian_users_items_limpio.csv') # Se carga en el dataframe nuestro archivo de items
df_User_Review =pd.read_csv('..\\..\\Archivos Limpios\\australian_user_review_limpio.csv') # Se carga en el dataframe nuestro archivo de reviews

In [26]:
# Por temas de recursos computacionales, se hace una muestra de 10000 registros
df_Steam_Games = df_Steam_Games.head(10000)
df_Users_Items = df_Users_Items.head(10000)
df_User_Review = df_User_Review.head(10000)

In [27]:
# En este apartado se trabaja la columna "posted", para convertir sus valores en fecha y eliminar aquellos registros que no sean fiables para el análisis
df_Steam_Games['release_date'] = df_Steam_Games['release_date'].astype(str).str.replace("Posted ", '') # Se aplica la sistitución de la palabra posted por nada
df_Steam_Games['release_date'] = df_Steam_Games['release_date'].astype(str).str.replace(",", '') # Se aplica la sistitución de la coma por nada
df_Steam_Games['release_date'] = df_Steam_Games['release_date'].astype(str).str.replace(".", '') # Se aplica la sistitución de la coma por nada
df_Steam_Games['release_date'] = pd.to_datetime(df_Steam_Games['release_date'], errors='coerce') # Se convierten los valores obtenidos en un formato tipo fecha
df_Steam_Games = df_Steam_Games.loc[df_Steam_Games['release_date'].notna()] # Se eliminan los registros vacios de la columna posted
# Debido a que nuestra temporalidad de análisis será anual, se procede a modificar la columna posted para solo obtener el año
df_Steam_Games['year'] = df_Steam_Games['release_date'].dt.year.astype('Int64')

# Rutinas

1. PlayTimeGenre

In [29]:
# Se unifican dos dataframe para obtener la información
df_Unido = pd.merge(df_Users_Items, df_Steam_Games[['item_id', 'year', 'genres']], on='item_id', how='inner')

#Agrupamos por genero y año de lanzamiento
df_Unido = df_Unido.groupby(['year', 'genres'])['playtime_forever'].sum().reset_index()

# Encontrar el género con más horas jugadas por año 
HorasMaximas = df_Unido.loc[df_Unido.groupby(['year', 'genres'])['playtime_forever'].idxmax()]

# Ordenar el DataFrame resultante
HorasMaximas = HorasMaximas.sort_values(by=['genres', 'playtime_forever', 'year'], ascending=[True, False, True])

# Guardar el resultado en un archivo CSV
HorasMaximas.to_csv('PlayTimeGenre.csv', index=False)

2. UserForGenre

In [33]:
# Se crea un nuevo dataframe más optimo para juegos
df_game = df_Steam_Games.drop(['name', 'developer', 'tags','specs'],axis=1)

# Merge de los DataFrames
merged_data=pd.merge(df_Users_Items,df_game,left_on='item_id', right_on='item_id', how='right')

merged_data.drop(['item_id'], axis=1, inplace=True)
merged_data.dropna(inplace=True)

merged_data['playtime_forever'] = merged_data['playtime_forever'].astype(int)
merged_data['year'] = merged_data['year'].astype(int)

#Encontrar el jugador con más horas jugadas por género y año
max_hours_df = merged_data.loc[merged_data.groupby(['genres', 'year'])['playtime_forever'].idxmax()]

#Filtrar las columnas necesarias
max_hours_df = max_hours_df[['genres', 'year', 'user_id', 'playtime_forever']]

#Segunda agrupación para obtener las horas jugadas por año para el usuario seleccionado
max_hours = max_hours_df.groupby(['genres', 'year', 'user_id'])['playtime_forever'].sum().reset_index()

max_hours.to_csv('UserForGenre.csv', index=False)

3. UsersRecommend

In [35]:
# Merge de los DataFrames
df_merged = pd.merge(df_User_Review, df_Steam_Games, on='item_id', how='left')

# Filtrar las filas que cumplen con las condiciones
result = df_merged.loc[(df_merged['recommend'] == True) & (df_merged['sentiment_analysis'].isin([1, 2])),
                           ['year_x', 'name']]

# Renombrar la columna 'year_x' a 'year'
result = result.rename(columns={'year_x': 'year'})

# Agrupar por 'year' y 'name' y contar las ocurrencias
grouped_result = result.groupby(['year', 'name']).size().reset_index(name='count')

# Ordenar por 'year' y 'count' en orden descendente
grouped_result = grouped_result.sort_values(by=['year', 'count'], ascending=[False, False])

# Obtener el top 3 por año
top3_by_year = grouped_result.groupby('year').head(3)

# Guardar el resultado en un archivo CSV
top3_by_year.to_csv('UsersRecommend.csv', index=False)

4. UsersWorstDeveloper

In [38]:
# Unir los DataFrames
df_merged = pd.merge(df_User_Review, df_Steam_Games, on='item_id', how='left')

# Filtrar las filas que cumplen con las condiciones
df_filtered = df_merged.loc[(df_merged['recommend'] == False) & (df_merged['sentiment_analysis'] == 0),
                           ['year_x', 'developer']]
 
# Renombrar la columna 'year_x' a 'year'
df_filtered = df_filtered.rename(columns={'year_x': 'year'})

# Contar las ocurrencias de cada desarrolladora por año
developer_counts = df_filtered.groupby(['year', 'developer']).size().reset_index(name='count')
    
# Ordenar por 'year' y 'count' en orden descendente
grouped_result = developer_counts.sort_values(by=['year', 'count'], ascending=[False, False])    

# Obtener el top 3 por año
result_df = grouped_result.groupby('year').head(3)    

# Guardar el resultado en un archivo CSV
result_df.to_csv('UsersWorstDeveloper.csv', index=False)

5. SentimentAnalisys

In [40]:
# Merge de los DataFrames
merged_df = pd.merge(df_Steam_Games, df_User_Review, on='item_id', how='inner')

# Agrupar por desarrolladora y sentimiento, y contar las ocurrencias
result_df = merged_df.groupby(['developer', 'sentiment_analysis']).size().reset_index(name='count')

# Pivoteando el DataFrame para tener sentimientos como columnas
result_df = result_df.pivot(index='developer', columns='sentiment_analysis', values='count').reset_index()

# Renombrar las columnas según el formato deseado
result_df.columns = ['developer', 'Negative', 'Neutral', 'Positive']

# Rellenar NaN con 0
result_df = result_df.fillna(0)

# Convertir las columnas a tipo int
result_df[['Negative', 'Neutral', 'Positive']] = result_df[['Negative', 'Neutral', 'Positive']].astype(int)

# Guardar el resultado en un archivo CSV
result_df.to_csv('sentiment_analysis.csv', index=False)